In [124]:
import pandas as pd
import requests as re
from app.api import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [125]:
user_name = 'IngrownHair132'
league_name = 'Any Given Sunday'
year = 2025

# Sleeper_API

In [169]:
# Get user information (mine)
user = get_user(user_name)
user_id = user['user_id']

# Get user league information (mine)
league = get_leagues(user_id, 2025)
league_id = pd.DataFrame(league)['league_id'][0]

# Get league users
users = league_users(league_id)
user_info = pd.DataFrame(users)[['display_name', 'user_id']]

# Get league rosters
roster = get_rosters(league_id)

# players
players = get_players()

## league_table

In [178]:
overview = pd.merge(user_info, pd.DataFrame(roster)[['owner_id', 'metadata', 'players', 'starters', 'settings']],
         left_on='user_id', right_on='owner_id', how='left')

In [180]:
overview['points_for'] = overview['settings'].apply(lambda x: x['fpts'])
overview['points_against'] = overview['settings'].apply(lambda x: x['fpts_against'])
overview['wins'] = overview['settings'].apply(lambda x: x['wins'])
overview['losses'] = overview['settings'].apply(lambda x: x['losses'])
overview['ties'] = overview['settings'].apply(lambda x: x['ties'])

overview['streak'] = overview['metadata'].apply(lambda x: x['streak'])

overview['points_delta'] = overview['points_for'] - overview['points_against']

In [181]:
league_table = overview.sort_values(by=['wins', 'points_for'], ascending=False)[['display_name', 'streak', 'wins', 'losses', 'points_for', 'points_against', 'points_delta']]
league_table

,display_name,streak,wins,losses,points_for,points_against,points_delta
4,seifmahmoud,1W,9,1,1659,1200,459
1,BlaiseMaweedi,2W,7,3,1553,1366,187
6,ruids,1W,6,4,1266,1172,94
7,RoryQ94,2W,6,4,1213,1311,-98
3,youssefjerome,1L,5,5,1392,1316,76
0,IngrownHair132,1L,5,5,1246,1233,13
2,SeifMattar,5L,2,8,1110,1456,-346
5,Kiki99,10L,0,10,1078,1462,-384


## team_roster

In [174]:
player_df = pd.DataFrame(players)

In [ ]:
overview

,display_name,user_id,owner_id,metadata,players,starters,settings,points_for,points_against,wins,losses,ties,streak,points_delta
0,IngrownHair132,648891376750120960,648891376750120960,"{'allow_pn_scoring': 'on', 'record': 'WLWLLLWW...","[10229, 11564, 11581, 11584, 11604, 11627, 116...","[11564, 12529, 3198, 9493, 10229, 11604, 5859,...","{'fpts': 1246, 'fpts_against': 1233, 'fpts_aga...",1246,1233,5,5,0,1L,13
1,BlaiseMaweedi,649252413399728128,649252413399728128,"{'allow_pn_scoring': 'off', 'p_nick_5012': '',...","[11563, 11624, 11632, 12527, 12711, 1479, 4033...","[11563, 9509, 4034, 7526, 9754, 8130, 5850, 12...","{'fpts': 1553, 'fpts_against': 1366, 'fpts_aga...",1553,1366,7,3,0,2W,187
2,SeifMattar,649285413139214336,649285413139214336,"{'record': 'LLLWWLLLLL', 'streak': '5L'}","[11565, 11575, 11589, 12481, 12490, 12530, 146...","[3294, 6790, 12481, 8112, 4039, 1466, 8136, 58...","{'fpts': 1110, 'fpts_against': 1456, 'fpts_aga...",1110,1456,2,8,0,5L,-346
3,youssefjerome,649285927532851200,649285927532851200,"{'record': 'WLLWWLLWWL', 'streak': '1L'}","[10236, 12489, 12498, 12514, 3321, 4018, 4037,...","[6904, 9221, 4866, 7547, 6786, 5001, 9224, 125...","{'fpts': 1392, 'fpts_against': 1316, 'fpts_aga...",1392,1316,5,5,0,1L,76
4,seifmahmoud,649978366040072192,649978366040072192,"{'record': 'WWWWWWWWLW', 'streak': '1W'}","[10859, 11533, 11560, 11637, 12504, 12509, 125...","[4984, 9226, 8138, 7564, 2133, 10859, 9756, 50...","{'fpts': 1659, 'fpts_against': 1200, 'fpts_aga...",1659,1200,9,1,0,1W,459
5,Kiki99,650885692238409728,650885692238409728,"{'record': 'LLLLLLLLLL', 'streak': '10L'}","[10219, 11576, 11620, 11655, 12484, 12506, 125...","[6804, 10219, 12507, 0, 7525, 8110, 11620, 681...","{'fpts': 1078, 'fpts_against': 1462, 'fpts_aga...",1078,1462,0,10,0,10L,-384
6,ruids,721066999974830080,721066999974830080,"{'record': 'LWWLLWWWLW', 'streak': '1W'}","[10222, 11583, 11631, 11635, 11638, 12508, 125...","[4881, 8155, 12512, 7569, 11635, 5022, 11631, ...","{'fpts': 1266, 'fpts_against': 1172, 'fpts_aga...",1266,1172,6,4,0,1W,94
7,RoryQ94,740241023237779456,740241023237779456,"{'record': 'LWWLWWLLWW', 'streak': '2W'}","[11539, 11566, 12485, 12526, 12533, 2216, 4199...","[96, 8150, 7021, 6794, 9488, 4217, 8126, 12526...","{'fpts': 1213, 'fpts_against': 1311, 'fpts_aga...",1213,1311,6,4,0,2W,-98


In [ ]:
roster_all = pd.DataFrame(columns=user_info['display_name'], index=league[0]['roster_positions'])

In [197]:
overview['players']

0    [10229, 11564, 11581, 11584, 11604, 11627, 116...
1    [11563, 11624, 11632, 12527, 12711, 1479, 4033...
2    [11565, 11575, 11589, 12481, 12490, 12530, 146...
3    [10236, 12489, 12498, 12514, 3321, 4018, 4037,...
4    [10859, 11533, 11560, 11637, 12504, 12509, 125...
5    [10219, 11576, 11620, 11655, 12484, 12506, 125...
6    [10222, 11583, 11631, 11635, 11638, 12508, 125...
7    [11539, 11566, 12485, 12526, 12533, 2216, 4199...
Name: players, dtype: object